In [1]:
import math
import sys,os
import numpy as np
import pandas as pd

import pysal
sys.path.append('/Users/Ziqi/Desktop/developer/gwr/')
from gwr.gwr import GWR
from gwr.sel_bw import Sel_BW
from spglm.family import Gaussian, Poisson, Binomial
%load_ext line_profiler

## GA Data

In [2]:
data = pysal.open(pysal.examples.get_path('GData_utm.csv'))
y = np.array(data.by_col('PctBach')).reshape((-1,1))
fb  = np.array(data.by_col('PctFB')).reshape((-1,1))
pov = np.array(data.by_col('PctPov')).reshape((-1,1)) 
blk = np.array(data.by_col('PctBlack')).reshape((-1,1))
X = np.hstack([fb,pov,blk])
coords = list(zip(data.by_col('X'), data.by_col('Y')))

### Fixed Gaussian

In [3]:
%%time
bw = Sel_BW(coords,y,X,fixed=True,kernel="gaussian",Ziqi=False).search(search="golden_section")
print("optimal bw found:", bw)

Using current fit func
optimal bw found: 109647.51
CPU times: user 925 ms, sys: 14.4 ms, total: 939 ms
Wall time: 934 ms


In [4]:
%%time
bw = Sel_BW(coords,y,X,fixed=True,kernel="gaussian",Ziqi=True).search(search="golden_section")
print("optimal bw found:", bw)

Using Ziqi's fast_search fit
optimal bw found: 109647.51
CPU times: user 155 ms, sys: 7.52 ms, total: 162 ms
Wall time: 156 ms


In [5]:
print("speed-up: ",934/156,"times")

speed-up:  5.987179487179487 times


### Adaptive Bisquare

In [6]:
%%time
bw = Sel_BW(coords,y,X,fixed=False,kernel="bisquare",Ziqi=False).search(search="golden_section")
print("optimal bw found:", bw)

Using current fit func
optimal bw found: 117.0
CPU times: user 348 ms, sys: 8.68 ms, total: 357 ms
Wall time: 351 ms


In [7]:
%%time
bw = Sel_BW(coords,y,X,fixed=False,kernel="bisquare",Ziqi=True).search(search="golden_section")
print("optimal bw found:", bw)

Using Ziqi's fast_search fit
optimal bw found: 117.0
CPU times: user 96 ms, sys: 5.8 ms, total: 102 ms
Wall time: 99.6 ms


In [8]:
print("speed-up: ",351/99.6,"times")

speed-up:  3.524096385542169 times


## A bigger dataset: county-level

In [9]:
#Reading the data
df = pd.read_csv('/Users/Ziqi/Desktop/2016 Election Data/df_5k_z_add_3rd_party.csv')
#Converting things into matrices
y = df.z_new_pct_dem.as_matrix().reshape(-1,1)
X = df.iloc[:,7:].as_matrix()
k = X.shape[1]
u = df.proj_X
v = df.proj_Y
n = X.shape[0]
coords = list(zip(u,v))

### Fixed Gaussian

In [10]:
%%time
bw = Sel_BW(coords,y,X,fixed=True,kernel="gaussian",Ziqi=False).search(search="golden_section")
print("optimal bw found:", bw)

Using current fit func
optimal bw found: 156597.54
CPU times: user 6min 4s, sys: 16.5 s, total: 6min 20s
Wall time: 3min 46s


In [11]:
%%time
bw = Sel_BW(coords,y,X,fixed=True,kernel="gaussian",Ziqi=True).search(search="golden_section")
print("optimal bw found:", bw)

Using Ziqi's fast_search fit
optimal bw found: 156597.54
CPU times: user 34.2 s, sys: 4.73 s, total: 38.9 s
Wall time: 34.9 s


In [12]:
print("speed-up: ",226/34.9,"times")

speed-up:  6.4756446991404015 times


### Adaptive Bisquare

In [13]:
%%time
bw = Sel_BW(coords,y,X,fixed=False,kernel="bisquare",Ziqi=False).search(search="golden_section")
print("optimal bw found:", bw)

Using current fit func
optimal bw found: 136.0
CPU times: user 2min 34s, sys: 8 s, total: 2min 42s
Wall time: 1min 41s


In [14]:
%%time
bw = Sel_BW(coords,y,X,fixed=False,kernel="bisquare",Ziqi=True).search(search="golden_section")
print("optimal bw found:", bw)

Using Ziqi's fast_search fit
optimal bw found: 136.0
CPU times: user 14.1 s, sys: 1.22 s, total: 15.3 s
Wall time: 15.2 s


In [15]:
print("speed-up: ",101/15.2,"times")

speed-up:  6.644736842105264 times


In [ ]:
%lprun -f GWR.fit Sel_BW(coords,y,X,fixed=True,kernel="gaussian",Ziqi=False).search(search="golden_section")

Using current fit func


In [20]:
%lprun -f GWR._fast_search Sel_BW(coords,y,X,fixed=True,kernel="gaussian",Ziqi=True).search(search="golden_section")

Using Ziqi's fast_search fit
*** KeyboardInterrupt exception caught in code being profiled.